In [1]:
from e2e_sae.scripts.train_tlens_saes.run_train_tlens_saes import Config
from e2e_sae.scripts.train_tlens_saes.run_train_tlens_saes import main as run_training
from e2e_sae.scripts.train_tlens_saes.run_train_tlens_bayesian_saes import main as run_bayesian_training


/home/raymond/miniconda3/envs/mi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define a Config
The sample config below will train a single SAE on layer 6 of gpt2 using an e2e loss.

Note that this will take 10-11 hours on an A100 to run. See
[e2e_sae/scripts/train_tlens_saes/tinystories_1M_e2e.yaml](../e2e_sae/scripts/train_tlens_saes/tinystories_1M_e2e.yaml)
for a tinystories-1m config, or simply choose a smaller model to train on and adjust the
n_ctx and dataset accordingly (some other pre-tokenized datasets can be found [here](https://huggingface.co/apollo-research)).


In [3]:
config = Config(
    wandb_project="gpt2-e2e_play",
    wandb_run_name=None,  # If not set, will use a name based on important config values
    wandb_run_name_prefix="",
    seed=0,
    tlens_model_name="gpt2-small",
    tlens_model_path=None,
    n_samples=400_000,
    save_every_n_samples=None,
    eval_every_n_samples=40_000,
    eval_n_samples=500,
    log_every_n_grad_steps=20,
    collect_act_frequency_every_n_samples=40_000,
    act_frequency_n_tokens=500_000,
    batch_size=8,
    effective_batch_size=16,  # Number of samples before each optimizer step
    lr=5e-4,
    lr_schedule="cosine",
    min_lr_factor=0.1,  # Minimum learning rate as a fraction of the initial learning rate
    warmup_samples=20_000,  # Linear warmup over this many samples
    max_grad_norm=10.0,  # Gradient norms get clipped to this value before optimizer steps
    loss={
        # Note that "original acts" below refers to the activations in a model without SAEs
        "sparsity": {
            "p_norm": 1.0,  # p value in Lp norm
            "coeff": 1.5,  # Multiplies the Lp norm in the loss (sparsity coefficient)
        },
        "in_to_orig": None,  # Used for e2e+future recon. MSE between the input to the SAE and original acts
        "out_to_orig": None,  # Not commonly used. MSE between the output of the SAE and original acts
        "out_to_in": {
            # Multiplies the MSE between the output and input of the SAE. Setting to 0 lets us track this
            # loss during training without optimizing it
            "coeff": 0.0,
        },
        "logits_kl": {
            "coeff": 1.0,  # Multiplies the KL divergence between the logits of the SAE model and original model
        },
    },
    train_data={
        # See https://huggingface.co/apollo-research for other pre-tokenized datasets
        "dataset_name": "apollo-research/Skylion007-openwebtext-tokenizer-gpt2",
        "is_tokenized": True,
        "tokenizer_name": "gpt2",
        "streaming": True,
        "split": "train",
        "n_ctx": 1024,
    },
    eval_data={
        # By default this will use a different seed to the training data, but can be set with `seed`
        "dataset_name": "apollo-research/Skylion007-openwebtext-tokenizer-gpt2",
        "is_tokenized": True,
        "tokenizer_name": "gpt2",
        "streaming": True,
        "split": "train",
        "n_ctx": 1024,
    },
    saes={
        "retrain_saes": False,  # Determines whether to continue training the SAEs in pretrained_sae_paths
        "pretrained_sae_paths": None,  # Path or paths to pretrained SAEs
        "sae_positions": [  # Position or positions to place SAEs in the model
            "blocks.6.hook_resid_pre",
        ],
        "dict_size_to_input_ratio": 60.0,  # Size of the dictionary relative to the activations at the SAE positions
    },
)

## Train

In [2]:
run_bayesian_training("e2e_sae/scripts/train_tlens_saes/tinystories_1M_e2e.yaml")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: raymondl to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


2025-04-17 03:08:38 - INFO - wandb_project='tinystories-1m-2' wandb_run_name=None wandb_run_name_prefix='' seed=0 tlens_model_name='roneneldan/TinyStories-1M' tlens_model_path=None save_dir=PosixPath('/home/raymond/.temp/e2e_sae/e2e_sae/scripts/train_tlens_saes/out') n_samples=400000 save_every_n_samples=None eval_every_n_samples=40000 eval_n_samples=500 batch_size=20 effective_batch_size=20 lr=0.001 lr_schedule='cosine' min_lr_factor=0.1 warmup_samples=20000 cooldown_samples=0 max_grad_norm=1.0 log_every_n_grad_steps=20 collect_act_frequency_every_n_samples=40000 act_frequency_n_tokens=500000 loss=LossConfigs(sparsity=SparsityLoss(coeff=3.0, p_norm=1.0), in_to_orig=None, out_to_orig=None, out_to_in=OutToInLoss(coeff=0.0), logits_kl=LogitsKLLoss(coeff=1.0)) train_data=DatasetConfig(dataset_name='apollo-research/roneneldan-TinyStories-tokenizer-gpt2', is_tokenized=True, tokenizer_name='gpt2', streaming=True, split='train', n_ctx=512, seed=None, column_name='input_ids') eval_data=Dataset

Loaded pretrained model roneneldan/TinyStories-1M into HookedTransformer
Moving model to device:  cuda


Steps:   0%|          | 0/20000 [00:01<?, ?it/s]

Samples 20 Batch_idx 0 GradUpdates 1 Loss 83.99545


...

Samples 399600 Batch_idx 19979 GradUpdates 19980 Loss 2.74373


Steps: 100%|█████████▉| 19999/20000 [43:43<00:00,  7.88it/s]

Samples 400000 Batch_idx 19999 GradUpdates 20000 Loss 2.71305


Eval Steps: 100%|██████████| 25/25 [00:04<00:00,  5.80it/s]
2025-04-17 03:52:32 - INFO - Saved config to /home/raymond/.temp/e2e_sae/e2e_sae/scripts/train_tlens_saes/out/seed-0_lpcoeff-3.0_logits-kl-1.0_lr-0.001_ratio-50.0_blocks.4.hook_resid_pre_2025-04-17_03-08-42/final_config.yaml
2025-04-17 03:52:32 - INFO - Saved model to /home/raymond/.temp/e2e_sae/e2e_sae/scripts/train_tlens_saes/out/seed-0_lpcoeff-3.0_logits-kl-1.0_lr-0.001_ratio-50.0_blocks.4.hook_resid_pre_2025-04-17_03-08-42/samples_400000.pt
Batches: 100%|██████████| 97/97 [00:05<00:00, 18.03it/s]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


grad_norm,█▅▄▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
grad_updates,▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇████
loss,█▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss/eval/in_to_orig/blocks.0.hook_resid_post,▁▁▁▁▁▁▁▁▁▁▁
loss/eval/in_to_orig/blocks.1.hook_resid_post,▁▁▁▁▁▁▁▁▁▁▁
loss/eval/in_to_orig/blocks.2.hook_resid_post,▁▁▁▁▁▁▁▁▁▁▁
loss/eval/in_to_orig/blocks.3.hook_resid_post,▁▁▁▁▁▁▁▁▁▁▁
loss/eval/in_to_orig/blocks.4.hook_resid_post,█▄▂▁▁▁▁▁▁▁▁
loss/eval/in_to_orig/blocks.5.hook_resid_post,█▄▂▁▁▁▁▁▁▁▁
loss/eval/in_to_orig/blocks.6.hook_resid_post,█▄▂▁▁▁▁▁▁▁▁
loss/eval/in_to_orig/blocks.7.hook_resid_post,█▄▂▁▁▁▁▁▁▁▁
